# Exposure bracketing using the sequencer feature of USB3 ace2devices

Documentation for sequencer feature on Ace2 and Boost-R is in https://docs.baslerweb.com/sequencer-(ace-2-and-boost-r)

In [1]:
import pypylon.pylon as py
import numpy as np

In [2]:
# open the first USB device
info = py.DeviceInfo()
info.SetDeviceClass("BaslerUsb")
cam = py.InstantCamera(py.TlFactory.GetInstance().CreateFirstDevice(info))

In [3]:
# this code only works for ace USB
if not cam.GetDeviceInfo().GetModelName().startswith("a2A"):
    print("_This_ sequencer configuration only works to basler ace2 USB")
    

In [4]:
# open device
cam.Open()

## setup camera.
sequencer mode is not running if the auto-functions are active

In [5]:
# reset to known state
cam.SequencerMode = "Off"
cam.SequencerConfigurationMode = "Off"
cam.UserSetSelector = "Default"
cam.UserSetLoad.Execute()

In [6]:
exp_0 = 100
exp_1 = 200
exp_2 = 300
exp_3 = 400

### activate chunks [ embedded data ]
This allows to read the exposuretime an image has been taken with from the image data

In [7]:
# enable camera chunk mode
cam.ChunkModeActive = True
# enable exposuretime chunk
cam.ChunkSelector = "ExposureTime"
cam.ChunkEnable = True
# enable sequencer set info
cam.ChunkSelector = "SequencerSetActive"
cam.ChunkEnable = True

### sequencer setup

The configuration in this example will switch to the next sequencer set for each image

* exp_0
* exp_1
* exp_2
* exp_3
* exp_0
* exp_1
* ...
  

In [8]:
cam.SequencerMode = "Off"
cam.SequencerConfigurationMode = "On"

### setup set 0

In [9]:
cam.ExposureTime = exp_0

# switch to the next set after image taken
cam.SequencerPathSelector = 0
cam.SequencerSetNext = 1
cam.SequencerTriggerSource = "ExposureStart"

cam.SequencerSetSelector = 0
cam.SequencerSetSave.Execute();

### setup set 1

In [10]:
cam.ExposureTime = exp_1

# switch to the next set after image taken
cam.SequencerPathSelector = 0
cam.SequencerSetNext = 2
cam.SequencerTriggerSource = "ExposureStart"

cam.SequencerSetSelector = 1
cam.SequencerSetSave.Execute();

### setup set 2

In [11]:
cam.ExposureTime = exp_2

# switch to the next set after image taken
cam.SequencerPathSelector = 0
cam.SequencerSetNext = 3
cam.SequencerTriggerSource = "ExposureStart"

cam.SequencerSetSelector = 2
cam.SequencerSetSave.Execute();

### setup set 3

In [12]:
cam.ExposureTime = exp_3

# switch to the _first_ set after image taken
cam.SequencerPathSelector = 0
cam.SequencerSetNext = 0
cam.SequencerTriggerSource = "ExposureStart"

cam.SequencerSetSelector = 3
cam.SequencerSetSave.Execute();

### Dump Configuration

In [13]:
for seq_set in range(cam.SequencerSetSelector.Max + 1):
    cam.SequencerSetSelector = seq_set
    cam.SequencerSetLoad.Execute()
    if cam.SequencerTriggerSource.Value == "Off":
        continue
    cam.SequencerPathSelector = 0
    next_set = cam.SequencerSetNext.Value
    print(f"set {seq_set}: -> {next_set} ( {cam.SequencerTriggerSource.Value} )")
    print("Exposure time", cam.ExposureTime.Value)
    
        

set 0: -> 1 ( ExposureStart )
Exposure time 100.0
set 1: -> 2 ( ExposureStart )
Exposure time 200.0
set 2: -> 3 ( ExposureStart )
Exposure time 300.0
set 3: -> 0 ( ExposureStart )
Exposure time 400.0


### exit configuration mode

In [14]:
cam.SequencerConfigurationMode = "Off"

### activate sequencer mode

In [15]:
# this will set the first sequencer set to set _0_ as side effect
cam.SequencerMode = "On"

### test capture with enabled sequencer mode

In [16]:
# grab 4 sets of 3 images
cam.StartGrabbingMax( 4 * 3)

while cam.IsGrabbing():
    with cam.RetrieveResult(1000) as res:
        exp_time_chunk = res.ChunkDataNodeMap.ChunkExposureTime.Value
        sequence_id_chunk = res.ChunkDataNodeMap.ChunkSequencerSetActive.Value
        print("%d\t%d\t%6.0f\t%6.2f" %(res.BlockID, sequence_id_chunk, exp_time_chunk, np.mean(res.Array)))
        # ... do something with the images
    
cam.StopGrabbing()

0	0	   100	  0.00
1	1	   200	  0.00
2	2	   300	  0.00
3	3	   400	  0.00
4	0	   100	  0.00
5	1	   200	  0.00
6	2	   300	  0.00
7	3	   400	  0.00
8	0	   100	  0.00
9	1	   200	  0.00
10	2	   300	  0.00
11	3	   400	  0.00


### Shutdown session after acquisition

This is required to rerun the notebook

In [17]:
cam.SequencerMode = "Off"

In [18]:
cam.Close()